In [1]:
import nltk
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
import requests
import glob
import os
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
import string
import re
import openpyxl
from openpyxl import Workbook,load_workbook

In [2]:

df = pd.read_excel('input.xlsx')
print(df)
stop_words = []
final_stop_words = []


      URL_ID                                                URL
0      123.0  https://insights.blackcoffer.com/rise-of-telem...
1      321.0  https://insights.blackcoffer.com/rise-of-e-hea...
2     2345.0  https://insights.blackcoffer.com/rise-of-e-hea...
3     4321.0  https://insights.blackcoffer.com/rise-of-telem...
4      432.0  https://insights.blackcoffer.com/rise-of-telem...
..       ...                                                ...
109  50921.0  https://insights.blackcoffer.com/coronavirus-i...
110  51382.8  https://insights.blackcoffer.com/coronavirus-i...
111  51844.6  https://insights.blackcoffer.com/what-are-the-...
112  52306.4  https://insights.blackcoffer.com/marketing-dri...
113  52768.2  https://insights.blackcoffer.com/continued-dem...

[114 rows x 2 columns]


In [3]:
# Define the directory path where the text files are located
dir = 'C:\\Users\\irzak\\ml\\intern'

In [5]:
# Use the glob module to retrieve a list of text files in the directory
file_paths = glob.glob(dir + '/*.txt')

for f in file_paths:
    with open(f, 'r') as file:
        content = file.read()
        words = content.split()
        stop_words.extend(words)

for x in stop_words:
    final_stop_words.append(x.lower())

final_stop_words = set(final_stop_words)
def remove_unicode(scrapped_data):
    no_unicode = []
    for i in scrapped_data:
        strencode = i.encode("ascii", "ignore")
        strdecode = strencode.decode()
        no_unicode.append(strdecode)
        
    return no_unicode

for ind in df.index:
    
    URL_ID = df['URL_ID'][ind]
    URL = df['URL'][ind]
    url = URL
    
    html = requests.get(url)
    
    soup = BeautifulSoup(html.content,'html.parser') 
    
   
    title = soup.find_all('h1')
    result_1 = soup.find('div',class_=['td-post-content', 'tagdiv-type'])
    
    if result_1 != None and len(title) >= 1:     
        
        para = result_1.find_all('p')
        heading = str(title[0].text)
        
        paragraph=""
       
        for p in para:
            paragraph = paragraph + str(p.text)
            
        
        scrapped_data = heading + paragraph
        scrapped_data = scrapped_data.split()
        scrapped_data = remove_unicode(scrapped_data)
        scrapped_data = ' '.join(scrapped_data)
        
        
        file_name = str(URL_ID)+'.txt'
        with open('C:\\Users\\irzak\\ml\\intern\\extracted\\'+file_name,'w') as file:
            words = scrapped_data.split()
            for word in words:
                file.write(word + ' ')
                    
                    
        file.close()  

KeyboardInterrupt: 

In [6]:
def tokenize_fun(extracted_str):
    text = extracted_str
    tkn = word_tokenize(text)
    return tkn
def tokenize_fun_sent(extracted_str):
    text = extracted_str
    tkn = sent_tokenize(text)
    return tkn
def filter_stopwords(word_tokens):
    filtered_data = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_data.append(w)
            
    return filtered_data

In [7]:
positive_words = set()
with open('C:\\Users\\irzak\\ml\\intern\\MasterDictionary\\positive-words.txt','r') as file:   
    for line in file:       
        for word in line.split():    
            positive_words.add(word)

In [8]:
negative_words = set()
with open('C:\\Users\\irzak\\ml\\intern\\MasterDictionary\\negative-words.txt','r') as file:   
    for line in file:       
        for word in line.split():    
            negative_words.add(word)

In [9]:
def filter_symbols(filtered_data):
    english_words = []
    words = set(nltk.corpus.words.words())

    for i in filtered_data:
        i = " ".join(w for w in nltk.wordpunct_tokenize(i) if w.lower() in words or not w.isalpha())
        english_words.append(i);
        
    english_words_no_punctuations = []
    for i in english_words:
        i = i.translate(str.maketrans('', '', string.punctuation))
        english_words_no_punctuations.append(i)
    
    no_unicode = []
    for i in english_words_no_punctuations:
        strencode = i.encode("ascii", "ignore")
        strdecode = strencode.decode()
        no_unicode.append(strdecode)
    
    english_words_no_punctuations = no_unicode    
    final_filtered_data = set(english_words_no_punctuations)
    final_filtered_data = list(final_filtered_data)
    for i in final_filtered_data:
        if i=='':
            final_filtered_data.remove(i)
        
    return final_filtered_data

In [10]:
def filter_inbuilt_stopwords_punctuations(word_tokens):
    inbuilt_stopwords = stopwords.words('english')
    
    data_no_stopwords = []
    for item in word_tokens:
        if item not in inbuilt_stopwords:
            data_no_stopwords.append(item)
            
    no_pun_stop = []
    for i in data_no_stopwords:
        i = i.translate(str.maketrans('', '', string.punctuation))
        no_pun_stop.append(i)
        
    return no_pun_stop

In [11]:
def syllable_count(word):
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e") or word.endswith("es") or word.endswith("ed"):
        count -= 1
        
    if word.endswith("ted") or word.endswith("tes") or word.endswith("ses") or word.endswith("ied") or word.endswith("ies"):
        count += 1
                
    if count == 0:
        count += 1
    return count

In [12]:
df_o = pd.DataFrame(columns=['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX_WORD_COUNT','WORD_COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])

In [13]:
# Function to calculate the number of syllables in a word
def syllable_count(word):
    # Your syllable counting logic here (you may use a library like Pyphen)
    # This is a simple example that assumes each vowel contributes one syllable
    vowels = "aeiouy"
    count = 0
    for char in word:
        if char.lower() in vowels:
            count += 1
    return max(count, 1)

# Function to calculate the FOG index
def calculate_fog_index(average_sentence_length, percentage_complex_words):
    return 0.4 * (average_sentence_length + percentage_complex_words)

# Define the directory path where the text files are located
directory = r'C:\Users\irzak\ml\intern\extracted'

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame(columns=[
    'URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 
    'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 
    'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 
    'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'
])

# Assuming you have a DataFrame df with 'URL_ID' and 'URL' columns
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    # Construct the file path based on URL_ID
    file_path = os.path.join(directory, f"{url_id}.txt")

    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File not found for URL_ID {url_id}")
        continue

    # Read the content of the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()

    # Tokenize the text into words and sentences
    word_tokens = word_tokenize(text_content)
    sentence_tokens = sent_tokenize(text_content)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in word_tokens if word.lower() not in stop_words and word.isalpha()]

    # Calculate scores based on your requirements
    positive_score = sum(1 for word in filtered_words if word in positive_words)
    negative_score = sum(1 for word in filtered_words if word in negative_words)
    
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / len(word_tokens) + 0.000001

    average_sentence_length = len(word_tokens) / len(sentence_tokens)
    percentage_complex_words = sum(1 for word in filtered_words if syllable_count(word) > 2) / len(word_tokens)
    
    fog_index = calculate_fog_index(average_sentence_length, percentage_complex_words)
    
    average_no_of_words_per_sentence = len(word_tokens) / len(sentence_tokens)
    
    complex_word_count = sum(1 for word in filtered_words if syllable_count(word) > 2)
    word_count = len(filtered_words)
    
    syllable_per_word = sum(syllable_count(word) for word in filtered_words) / len(filtered_words)
    
    personal_pronouns = len(re.findall(r'\b(he|she|it|we|they|you|I)\b', text_content, re.IGNORECASE))
    
    total_characters = sum(len(word) for word in filtered_words)
    average_word_length = total_characters / len(filtered_words)

    # Append the results to the DataFrame
    row_data = {
    'URL_ID': url_id,
    'URL': url,
    'POSITIVE SCORE': positive_score,
    'NEGATIVE SCORE': negative_score,
    'POLARITY SCORE': polarity_score,
    'SUBJECTIVITY SCORE': subjectivity_score,
    'AVG SENTENCE LENGTH': average_sentence_length,
    'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
    'FOG INDEX': fog_index,
    'AVG NUMBER OF WORDS PER SENTENCE': average_no_of_words_per_sentence,
    'COMPLEX WORD COUNT': complex_word_count,
    'WORD COUNT': word_count,
    'SYLLABLE PER WORD': syllable_per_word,
    'PERSONAL PRONOUNS': personal_pronouns,
    'AVG WORD LENGTH': average_word_length
}

    result_df = pd.concat([result_df, pd.DataFrame([row_data])], ignore_index=True)

    # Save the results to an Excel file
    result_df.to_excel('output_results.xlsx', index=False)


File not found for URL_ID 11668.0
File not found for URL_ID 17671.4


In [14]:
jupyter nbconvert --to script your_notebook.ipynb

SyntaxError: invalid syntax (2621717190.py, line 1)